In [3]:
with open(f"./small_dgemv_naive_c_implementation_ver1-0_XmmNaive.c", "w") as p:

    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("#include <xmmintrin.h>\n")     

    code = f"""        
double sum_xmm_elements(__m128d xmmX) {{
    __m128d xmm_high_low = _mm_add_pd(xmmX, _mm_unpackhi_pd(xmmX, xmmX));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}}    
    

void mydgemv_n_ver1_0(double a[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m128d xmm0, xmm1, xmm2;
    double tmp_x;
    for (int64_t i=0; i<lda; i++){{
        y[i] = 0.0;
    }}
    
    int64_t i_remain = lda % 2;
    for (int64_t k=0; k<ldx; k++){{
        xmm0 = _mm_set1_pd(x[k]);        
        for (int64_t i=0; i<lda-i_remain; i+=2){{
            xmm1 = _mm_loadu_pd(&y[i]);
            xmm2 = _mm_loadu_pd(&a[i]);
            
            xmm2 = _mm_mul_pd(xmm0, xmm2);
            
            xmm1 = _mm_add_pd(xmm1, xmm2);
            
            _mm_storeu_pd(&y[i], xmm1);
        }}
        
        
        for (int64_t i=lda-i_remain; i<lda; i++){{
            y[i] += a[i] * x[k];
        }}         
        a += lda;
    }}
}}


void mydgemv_t_ver1_0(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m128d xmm0, xmm1, xmm2;
    
    for (int64_t i=0; i<lda; i++){{
        xmm0 = _mm_setzero_pd();
        
        for (int64_t k=0; k<ldx; k+=2){{
            xmm1 = _mm_loadu_pd(&x[k]);
            xmm2 = _mm_loadu_pd(&a_t[k]);
            
            xmm1 = _mm_mul_pd(xmm1, xmm2);
            
            xmm0 = _mm_add_pd(xmm0, xmm1);
        }}
        y[i] = sum_xmm_elements(xmm0);
        a_t += ldx;
    }}
}}
    """
    p.write(code)